# Assignment 1 -- Naive Bayes and K-Nearest Neighbour for Predicting Stroke

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier

#All functions that will be used
def preprocess(csv):
    return pd.read_csv(csv)

def split_data(table):
    #30% for test set, 70% for train set
    # random seed is 7, to keep the only one result in this assignment
    return train_test_split(table.iloc[:,: 10], table.iloc[:,-1],test_size=0.3, random_state=7) 

def train(x_train, y_train):
    train_set = pd.concat([x_train, y_train], axis = 1)
    p_dict = {}
    labels = x_train.columns
    y_instances = y_train.value_counts().items()
    
    #How many features for each label, otherwise work_type would give wierd error when stroke is 1, so define it at first
    features = {}
    for j in labels:
        features[j] = x_train[j].value_counts().index
    
    aset = train_set.groupby(y_train.values)

    for y_item in y_instances:
        # prob for each label when stroke = 0 and 1
        p_dict[y_item[0]] = {}
        
    for temp in aset:
        for af in labels:
            each_p = {}
            instance_f = temp[1][af].value_counts()
            #work_type class is 3 when stroke is 1, so add 0 to match the length
            for af_type in features[af]:
                if not(af_type in instance_f):
                    instance_f[af_type] = 0
                    
            for i_f_item in instance_f.items():
                #Laplace smoothing, numerator is added with alpha(normally 1) and denominator becomes M*1+count.
                p_f = (i_f_item[1] + 1) / (len(temp[1][af]) + len(features[af]))
                each_p[i_f_item[0]] = p_f
                
            p_dict[temp[0]][af] = each_p
    return p_dict
   
def predict(adict, x_test, y_train):
    #get the probabilities of storke 0 and 1
    py=getYprob(y_train)
    prelist=[]
    #for each instance in test set
    for test_item in range(len(x_test)):
        item = x_test.iloc[test_item]
        mrate=0
        y_c = ""
        #at the second dict which contains value
        for aclass in adict.items():
            # for stroke 0 or 1
            for i in range(len(py)):
                # for each data in a label
                for a in aclass[1].items():
                    #calculate rate
                    r = py[i] * a[1].get(item[a[0]])
                # situation1: mrate is 0 at the beginning
                if mrate == 0:
                    mrate=r
                    y_c = aclass[0]
                # situation2: when mrate is not 1
                if r > mrate:
                    mrate=r
                    y_c = aclass[0]
        prelist.append(y_c)
    
    return prelist

def evaluate(prelist, y_test):
    return metrics.accuracy_score(y_test.values, prelist), metrics.f1_score(y_test.values, prelist), metrics.recall_score(y_test.values, prelist), metrics.precision_score(y_test.values, prelist)
    
def replace_to_number(table):
    l1st = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
    for item in l1st:
        el = np.unique(table[item])
        lens = len(el)
        for i in range(lens):
            table[item].replace(el[i], i, inplace=True)
    return table

def discrete(table):
    for a_label in ['avg_glucose_level','bmi','age']:
        table[a_label] = pd.cut(table[a_label], 10, labels=range(10))
    return table

def getYprob(y):
    p_y = []
    for y_one in y.value_counts().sort_index(ascending=True).items():
        p_y.append(y_one[1]/len(y))
    return p_y

def knn(x_train, y_train, kn):
    classifier = KNeighborsClassifier(n_neighbors = kn)
    classifier.fit(x_train, y_train)
    return classifier


# Question 1

<font size=4>**a). Explore the data and summarise different aspects of the data. Can you see any interesting
characteristic in features, classes or categories? What is the main issue with the data?
Considering the issue, how would the Naive Bayes classifier work on this data? Discuss
your answer based on the Naive Bayes’ formulation (3 marks)**</font>

In [66]:
table = preprocess("stroke_update.csv")
age_row = table.iloc[:, 2]
table

,avg_glucose_level,bmi,age,gender,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,stroke
0,137.45,26.0,53,Male,0,0,Yes,Self-employed,Rural,smokes,0
1,56.85,24.4,44,Female,0,0,Yes,Private,Rural,never smoked,0
2,87.79,41.1,49,Female,0,0,No,Private,Urban,never smoked,0
3,94.82,22.9,28,Female,0,0,No,Private,Urban,never smoked,0
4,96.80,29.6,73,Male,0,0,Yes,Private,Urban,formerly smoked,0
...,...,...,...,...,...,...,...,...,...,...,...
2735,88.29,36.0,79,Male,0,1,Yes,Self-employed,Urban,never smoked,1
2736,93.38,26.7,76,Male,0,0,Yes,Self-employed,Rural,formerly smoked,1
2737,83.27,32.9,56,Female,0,0,Yes,Private,Rural,smokes,1
2738,75.91,26.7,80,Female,0,0,Yes,Self-employed,Urban,never smoked,1


**Summarise:**

<font size=3>
Overall there are 2740 instances and 11 attributes. The avg_glucose_level, bmi and age are numeric while others are not. The avg_glucose_level, bmi are continuous while others are discrete. Some features are in letters which makes code difficult to do analysis(e.g. ever_married, work_type etc) but some other features are in number(e.g. hypertension and heart_disease). The table is not randomly ordered for stroke starts from (stroke=0)  then followed by (stroke=1).
</font>

**Interesting characteristic:**

<font size=3>
From 0 to 2734, the stroke are all 0 and from 2735 to the end, the strokes are all 1. The test set and training set both require that each instance is randomly chosen.
</font>

In [65]:
new_table = replace_to_number(table)
new_table.corr()

,avg_glucose_level,bmi,age,gender,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,stroke
avg_glucose_level,1.000000,0.222221,0.267942,0.048000,0.153631,0.199533,0.140880,0.027007,-0.010321,-0.075984,0.195651
bmi,0.222221,1.000000,0.073568,0.015591,0.120742,0.015180,0.130180,-0.058395,0.015028,-0.035732,-0.010339
age,0.267942,0.073568,1.000000,0.062799,0.299846,0.261760,0.525890,0.033891,0.016144,-0.188859,0.431107
gender,0.048000,0.015591,0.062799,1.000000,0.039282,0.102391,0.051499,0.038683,0.021190,-0.030364,0.035096
hypertension,0.153631,0.120742,0.299846,0.039282,1.000000,0.127824,0.141752,0.022000,-0.000873,-0.053598,0.192728
heart_disease,0.199533,0.015180,0.261760,0.102391,0.127824,1.000000,0.100569,0.006069,-0.006143,-0.035016,0.256132
ever_married,0.140880,0.130180,0.525890,0.051499,0.141752,0.100569,1.000000,-0.058714,0.027941,-0.059003,0.158306
work_type,0.027007,-0.058395,0.033891,0.038683,0.022000,0.006069,-0.058714,1.000000,-0.006563,-0.044576,0.052052
Residence_type,-0.010321,0.015028,0.016144,0.021190,-0.000873,-0.006143,0.027941,-0.006563,1.000000,-0.032341,0.003285
smoking_status,-0.075984,-0.035732,-0.188859,-0.030364,-0.053598,-0.035016,-0.059003,-0.044576,-0.032341,1.000000,-0.057498


**Main Issue**

<font size=3>
According to the output above, some results are high which means they have correlation.Based on Naive Bayers, P(x1, x2, ..., xm |y)P(y) ≈ P(x1|y)P(x2|y)...P(xm|y)P(y), under the condition of class y, it is assumed that the features are independent. In this case, some features have strong correlation(e.g. age and ever_married) and the formular equation would not hold anymore. So there is a good way to handle this data set that is delete the data that has strong correlation or join them together.
</font>

<font size=4>**b). Is accuracy an appropriate metric to evaluate the models created for this data? Justify
your answer. Explain which metric(s) would be more appropriate, and contrast their utility
against accuracy. [no programming required] (2 marks)**</font>

<font size=3>
No

Only accuracy is not enough to evaluate the probaility of getting stroke for a person. Some people who have stroke would be diagnosed with no stroke and it would cause a serious consequence. There is no reason to gurantee that our data set is disaggregated. So accuracy is not full-scale. Precision and recall and F1-score would be more appropriate. Accuracy is only the frequence of the classifer is correct. Precision is how many people that are identified as stroke by classifer is actually identified as stroke. Recall is how many people are figured out with stroke in all the people that actually have stroke or not. F1-score is an additional opinion to evaluate the precision and recall better.
</font>

# Question 2

<font size=4>**a). Explain the independence assumption underlying Naive Bayes. What are the advantages
and disadvantages of this assumption? Elaborate your answers using the features of the
provided data. [no programming required] (1 mark)**</font>

**Explain:**

<font size=3>
The independence assumption is that assuming the features are strongly independent with each other. In the real life, that is not always true, so it is called "naive".
</font>

**Advantage:**

<font size=3>
It is easy to build and estimate model, for example, work_type has 5 types and ever_married has 2 types, the overall possibailities are too many and it makes joint probabilities really hard to calculate and there are also many 0 probabilities in it. So "naive" could help us solve the problems and it works well. 
</font>

**Disadvantage:**

<font size=3>
In real life, the "naive" assumption is not always true. For example, according to the output of "new_table.corr()" above, when age is 1, ever_marriged is 0.5258 which means they have strong correlation. But "naive" assumption ignore both weak(e.g. BMI=0.2 and age=1) and strong correlation and it is unreasonable.So it makes Naive Bayers naturally has an error.
</font>


<font size=4>**b). Implement the Naive Bayes classifier. You need to decide how you are going to apply
Naive Bayes for nominal and numeric attributes. You can combine both Gaussian and
Categorical Naive Bayes (option 1) or just using Categorical Naive Bayes (option 2). Explain
your decision.
For Categorical Naive Bayes, you can choose either epsilon or Laplace smoothing
for this calculation. Evaluate the classifier using accuracy and appropriate metric(s) on test
data. Explain your observations on how the classifiers have performed based on the metric(
s). Discuss the performance of the classifiers in comparison with the Zero-R baseline.
(4 marks)**</font>

**Only using Categotical Naive Bayers**

<font size=3>
According to the data below, the data is not suitable for Gaussian distribution. Because some features has a high maximum value and low minium value while the difference between mean with 2 values are high(e.g. bmi max: 70.3, bmi min: 11.0, bmi mean:  29.992043795620443). Using Categotical Naive Bayers can process data easier and it is a suitable way to do it.
</font>

In [336]:
print(table['work_type'].value_counts())
print("bmi max: ", table['bmi'].max())
print("bmi min: ", table['bmi'].min())
print("bmi mean: ", table['bmi'].mean())
print("agl max: ",table['avg_glucose_level'].max())
print("agl min: ",table['avg_glucose_level'].min())
print("agl mean: ",table['avg_glucose_level'].mean())
table

2    1742
3     577
0     366
4      49
1       6
Name: work_type, dtype: int64
bmi max:  70.3
bmi min:  11.0
bmi mean:  29.992043795620443
agl max:  271.74
agl min:  55.01
agl mean:  111.3860401459854


,avg_glucose_level,bmi,age,gender,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,stroke
0,137.45,26.0,53,1,0,0,1,3,0,2,0
1,56.85,24.4,44,0,0,0,1,2,0,1,0
2,87.79,41.1,49,0,0,0,0,2,1,1,0
3,94.82,22.9,28,0,0,0,0,2,1,1,0
4,96.80,29.6,73,1,0,0,1,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2735,88.29,36.0,79,1,0,1,1,3,1,1,1
2736,93.38,26.7,76,1,0,0,1,3,0,0,1
2737,83.27,32.9,56,0,0,0,1,2,0,2,1
2738,75.91,26.7,80,0,0,0,1,3,1,1,1


In [435]:
#This part is for train()
table = replace_to_number(preprocess("stroke_update.csv"))
x_train, x_test, y_train, y_test = split_data(table)
discrete_x_train = discrete(x_train)
train_dict = train(discrete_x_train, y_train)

<font size=3>---------------data above for supporting the partial answer b)(the one at top)-----------------------------------------</font>

<font size=3>---------------data below for supporting the other partial answer b)(the one at bottom)-----------------------------------------</font>

In [433]:
x_train['work_type'].value_counts()

2    1210
3     411
0     261
4      32
1       4
Name: work_type, dtype: int64

In [21]:
#This part is for predict()
table = replace_to_number(preprocess("stroke_update.csv"))
x_train, x_test, y_train, y_test = split_data(table)
discrete_x_train = discrete(x_train)
train_dict = train(discrete_x_train, y_train)
discrete_x_test = discrete(x_test)
print("Total num of test instance: ",len(predict(train_dict, discrete_x_test, y_train)))
#Each list represent the predict of one instance from all cases
prelist = predict(train_dict, discrete_x_test, y_train)
e = evaluate(prelist, y_test)
print("CNB accuracy: ", e[0])
print("CNB F1-Score: ", e[1])
print("CNB recall: ", e[2])
print("CNB precision: ", e[3])

Total num of test instance:  822
CNB accuracy:  0.6581508515815085
CNB F1-Score:  0.28498727735368956
CNB recall:  0.32
CNB precision:  0.25688073394495414


In [424]:
#Zero-R
print(y_test.value_counts())
zeroRlist=[0 for _ in range(len(x_test))]
print("Zero-R accuracy: ",metrics.accuracy_score(y_test.values, zeroRlist))
print("Zero-R F1-Score: ",metrics.f1_score(y_test.values, zeroRlist))
print("Zero-R recall: ",metrics.recall_score(y_test.values, prelist))
#precision is actually 0, using code will get warning
print("Zero-R precision: ", 0.0)
# print(metrics.precision_score(y_test.values, prelist))

0    660
1    162
Name: stroke, dtype: int64
Zero-R accuracy:  0.8029197080291971
Zero-R F1-Score:  0.0
Zero-R recall:  0.0
Zero-R precision:  0.0


**Observations**

<font size=3>
In this case, aloungh the accuracy of CNB is very high, but it is not good enough as a metric. Because Zero-R also has a high accuracy while other metrics(e.g. precision, recall, F1-Score) are all 0. So the performance of accuracy is really bad. The combination of accuracy, precision, recall and F1-Score could perform better. In this case, CNB F1-score only has 66%, and the value of recall(30.4%), precision(24.30%) are also low. It means the performance of perdict is not good as we expect. But accuracy looks like all goes well.
</font>

<font size=4>**c). Explain the difference between epsilon and Laplace smoothing. [no programming
required] (1 mark)**</font>

<font size=3>In Laplace smoothing, numerator is added with alpha(normally 1) and denominator becomes M * alpha + count. But in epsilon smoothing, 0 is only replaced with as much small constant as it can to keep probabilities with the most minimal change and the distribution is kept all the time. In Laplace smoothing, if the instances is few, the change would be drastical while the change would be small if there are few instances.</font>

# Question 3

<font size=4>**a) Implement the K-NN classifier, and find the optimal value for K. (1 mark)**</font>

In [414]:
#find the optimal value for K from[0, 1000]
knnlist=[]
for i in range(1, 201):
    knnlist.append(evaluate(knn(x_train, y_train, i).predict(x_test), y_test)[0])

#Because the index starts from 0, to match kn, it should plus 1, index 0 represent accuracy in evaluate
print("evaluate from loop kn=",knnlist.index(max(knnlist))+1,": ",max(knnlist))
#Check if the evaluate of two kn are same or not, index 0 represent accuracy in evaluate
print("evaluate from kn=129: ", evaluate(knn(x_train, y_train, 129).predict(x_test), y_test)[0])

evaluate from loop kn= 129 :  0.816301703163017
evaluate from kn=129:  0.816301703163017


<font size=3>
To sum up, according to the data above, the optimal value for K in [1,200] is 129.
</font>

<font size=4>**b) Based on the obtained value for K in question 4 (a), evaluate the classifier using accuracy
and chosen metric(s) on test data. Explain your observations on how the classifiers have
performed based on the metric(s). Discuss the performance of the classifiers in comparison
with the Zero- R baseline. (2 marks)**</font>

In [421]:
#Zero-R
print(y_test.value_counts())
zeroRlist=[0 for _ in range(len(x_test))]
print("Zero-R accuracy: ",metrics.accuracy_score(y_test.values, zeroRlist))
print("Zero-R F1-Score: ",metrics.f1_score(y_test.values, zeroRlist))
print("Zero-R recall: ",metrics.recall_score(y_test.values, prelist))
#precision is actually 0, using code will get warning
print("Zero-R precision: ", 0.0)

0    660
1    162
Name: stroke, dtype: int64
Zero-R accuracy:  0.8029197080291971
Zero-R F1-Score:  0.0
Zero-R recall:  0.0
Zero-R precision:  0.0


In [416]:
e = evaluate(knn(x_train, y_train, 129).predict(x_test), y_test)
print("k-nn accuracy: ", e[0])
print("k-nn F1-Score: ", e[1])
print("k-nn recall: ", e[2])
print("k-nn precision: ", e[3])

k-nn accuracy:  0.816301703163017
k-nn F1-Score:  0.32888888888888884
k-nn recall:  0.22839506172839505
k-nn precision:  0.5873015873015873


**Observation**

<font size=3>
 The accuracy of K-NN and Zero-R is close(80.29% for 0-R and 81.63% for K-NN).<br>
 58.73% people are predicted out in all people who have stroke. In 0-R, it's 0<br>
 22.84% people have stroke out of people predicted out.In 0-R, it's 0<br>
 F1-Score is 32.89%.In 0-R, it's 0<br>
 Accuracy is not enough to evaluate, same with comparing NB with Zero-R, the performance of accuracy is really bad. But F1-Score could represent the performance of precision and recall. Precision and recall also show that the predict result is not so good.
</font>

<font size=4>**c) Compare the classifiers (Naive Bayes and K-NN) based on metrics’ results. Provide a
comparatory discussion on the results. [no programming required] (1 mark)**</font>

<font size=3>The accuracy of Naive Bayes and K-NN is similiar, the result of F1-Score and accuracy is also closer. But KNN seems to have a higher predict performances on F1-Score and precision that are greater than those of NB classifier.<br>
For example, the comparison in KNN vs CNB: <br>
F1-Score: knn-32.89% vs cnb-28.50%<br>
recall:  knn22.82% vs cnb-32.00%<br>
precision:  knn58.73% vs cnb-25.69%<br>
</font>